In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1732394786212_0002,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@72acf28a


In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res2: org.apache.spark.SparkContext = org.apache.spark.SparkContext@13813f44


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW covid_data 
USING csv 
OPTIONS (path 's3://jupyterbuck/bigdata/datasets/covid.csv', header 'true', inferSchema 'true', encoding 'UTF-8');
"""
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.sql(
"""
SELECT *
FROM covid_data
LIMIT 1
"""
).columns
# Muestra una lista de los nombres de las columnas del DataFrame covid_data.


In [ ]:
spark.sql(
"""
SELECT *
FROM covid_data
LIMIT 20
"""
).show()


In [ ]:
len(spark.sql(
"""
SELECT *
FROM covid_data
LIMIT 1
"""
).columns)
# Retorna el número de columnas en el DataFrame covid_data.


In [ ]:
spark.sql(
"""
SELECT COUNT(*) as total_rows
FROM covid_data
"""
).show()
# Cuenta el número total de filas en el DataFrame covid_data.


In [ ]:
spark.sql(
"""
DESCRIBE TABLE covid_data
"""
).show(truncate=False)
# Imprime el esquema del DataFrame covid_data en un formato tabular.


In [ ]:
write_uri='s3a://jupyterbuck/bigdata/jupyter/spark/csv/'

In [ ]:
parquet_uri='s3a://jupyterbuck/bigdata/jupyter/spark/parket/'

In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW selected_data AS
SELECT `Nombre departamento`, `Nombre municipio`, Edad, Sexo
FROM covid_data
"""
)

# Mostrar los datos seleccionados
spark.sql("SELECT * FROM selected_data").show()

# Guardar en formato CSV en S3 con sobrescritura
spark.sql("SELECT * FROM selected_data").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "selected_data")

# Guardar en formato Parquet con sobrescritura
spark.sql("SELECT * FROM selected_data").write.format('parquet').mode("overwrite").save(parquet_uri + "selected_data")


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW covid_data_renamed AS
SELECT `Nombre Municipio` AS Municipio, `Nombre departamento` AS Departamento, *
FROM covid_data
"""
)
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW selected_data AS
SELECT Departamento, Municipio, Edad, Sexo
FROM covid_data_renamed
"""
)

# Mostrar los datos seleccionados
spark.sql("SELECT * FROM selected_data").show()

# Guardar en formato CSV en S3 con sobrescritura
spark.sql("SELECT * FROM selected_data").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "selected_data")

# Guardar en formato Parquet con sobrescritura
spark.sql("SELECT * FROM selected_data").write.format('parquet').mode("overwrite").save(parquet_uri + "selected_data")


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW grouped_departments AS
SELECT Departamento, COUNT(*) as count
FROM covid_data_renamed
GROUP BY Departamento
ORDER BY count DESC
"""
)

# Mostrar las primeras 10 filas
spark.sql("SELECT * FROM grouped_departments LIMIT 10").show()

# Guardar en formato CSV en S3 con sobrescritura
spark.sql("SELECT * FROM grouped_departments").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "grouped_departments_csv")

# Guardar en formato Parquet con sobrescritura
spark.sql("SELECT * FROM grouped_departments").write.format('parquet').mode("overwrite").save(parquet_uri + "grouped_departments_parquet")


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW grouped_municipios AS
SELECT Municipio, COUNT(*) as count
FROM covid_data_renamed
GROUP BY Municipio
ORDER BY count DESC
"""
)

# Mostrar las primeras 10 filas
spark.sql("SELECT * FROM grouped_municipios LIMIT 10").show()

# Guardar en formato CSV en S3 con sobrescritura
spark.sql("SELECT * FROM grouped_municipios").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "grouped_municipios_csv")

# Guardar en formato Parquet con sobrescritura
spark.sql("SELECT * FROM grouped_municipios").write.format('parquet').mode("overwrite").save(parquet_uri + "grouped_municipios_parquet")


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW grouped_ages AS
SELECT Edad, COUNT(*) as count
FROM covid_data_renamed
GROUP BY Edad
ORDER BY Edad ASC
"""
)

# Mostrar todas las filas del resultado
spark.sql("SELECT * FROM grouped_ages").show()

# Guardar en formato CSV en S3 con sobrescritura
spark.sql("SELECT * FROM grouped_ages").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "grouped_ages_csv")

# Guardar en formato Parquet con sobrescritura
spark.sql("SELECT * FROM grouped_ages").write.format('parquet').mode("overwrite").save(parquet_uri + "grouped_ages_parquet")


In [ ]:
# Importa las funciones lower y col para manipular las columnas
from pyspark.sql.functions import lower, col

# Convierte la columna "Sexo" a minúsculas
df = df.withColumn("Sexo", lower(col("Sexo")))
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW grouped_sex AS
SELECT Sexo, COUNT(*) as count, COUNT(*) / (SELECT COUNT(*) FROM covid_data_renamed) * 100 as Porcentaje
FROM covid_data_renamed
GROUP BY Sexo
"""
)
spark.sql("SELECT * FROM grouped_sex").show()
spark.sql("SELECT * FROM grouped_sex").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "grouped_sex_csv")
spark.sql("SELECT * FROM grouped_sex").write.format('parquet').mode("overwrite").save(parquet_uri + "grouped_sex_parquet")


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW antioquia_cases AS
SELECT *
FROM covid_data_renamed
WHERE Departamento = 'Antioquia'
"""
)

# Mostrar las primeras 20 filas del resultado
spark.sql("SELECT * FROM antioquia_cases").show()

# Guardar los datos filtrados en un archivo CSV en S3 con sobrescritura
spark.sql("SELECT * FROM antioquia_cases").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "antioquia_cases_csv")

# Guardar los datos filtrados en un archivo Parquet con sobrescritura
spark.sql("SELECT * FROM antioquia_cases").write.format('parquet').mode("overwrite").save(parquet_uri + "antioquia_cases_parquet")


In [ ]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW antioquia_elderly AS
SELECT *
FROM covid_data_renamed
WHERE Departamento = 'Antioquia' AND Edad > 60
"""
)

# Mostrar las primeras 20 filas del resultado
spark.sql("SELECT * FROM antioquia_elderly").show()

# Guardar los datos filtrados en un archivo CSV en S3 con sobrescritura
spark.sql("SELECT * FROM antioquia_elderly").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "antioquia_elderly_csv")

# Guardar los datos filtrados en un archivo Parquet con sobrescritura
spark.sql("SELECT * FROM antioquia_elderly").write.format('parquet').mode("overwrite").save(parquet_uri + "antioquia_elderly_parquet")


In [ ]:
from pyspark.sql.functions import when

df = df.withColumn(
    "Rango de Edad",
    when(df["Edad"] < 18, "Menor").when(df["Edad"] <= 60, "Adulto").otherwise("Mayor")
)

# Crear una vista temporal para los datos categorizados
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW categorized_data AS
SELECT *, CASE 
    WHEN Edad < 18 THEN 'Menor'
    WHEN Edad <= 60 THEN 'Adulto'
    ELSE 'Mayor'
END AS `Rango de Edad`
FROM covid_data_renamed
"""
)

# Mostrar las primeras 20 filas del nuevo DataFrame categorized_data
spark.sql("SELECT * FROM categorized_data").show()

# Guardar en formato CSV en S3 con sobrescritura
spark.sql("SELECT * FROM categorized_data").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(write_uri + "categorized_data")

# Guardar en formato Parquet con sobrescritura
spark.sql("SELECT * FROM categorized_data").write.format('parquet').mode("overwrite").save(parquet_uri + "categorized_data")
